References:

[Malayalam Baseline Model](https://kurianbenoy.com/ml-blog/malayalamtextmodels/malayalam/nlp/opensource/ml/deep%20learning/smc/2022/06/04/baselinemodel.html)

# 1. Prerequistes

In [ ]:
repo_id = "rajeshradhakrishnan/ml-news-classify-fastai"
#repo_id = "fastai X Hugging Face Group 2022/ml-news-classify"

In [6]:
# Installing from main and specifying ["fastai"] to automatically install the relevant packages and versions
# %%capture
!pip install -Uqq huggingface_hub["fastai"] datasets ohmeow-blurr
# not needed just for reference:
#!pip install -Uqq transformers transformers['sentencepiece'] datasets

     |████████████████████████████████| 81 kB 4.3 MB/s 
     |████████████████████████████████| 1.2 MB 11.2 MB/s 
     |████████████████████████████████| 43 kB 2.9 MB/s 
     |████████████████████████████████| 96 kB 7.2 MB/s 
     |████████████████████████████████| 4.4 MB 50.5 MB/s 
     |████████████████████████████████| 6.6 MB 52.9 MB/s 


In [ ]:
%%capture
!git clone https://github.com/ohmeow/blurr.git
%cd blurr
!pip install -e ".[dev]"

In [ ]:
import torch
import datasets
import transformers
from fastai.text.all import *

from blurr.text.data.all import *
from blurr.text.modeling.all import *

# 2. Prep Data

In [ ]:
raw_datasets = datasets.load_dataset("rajeshradhakrishnan/malayalam_news")

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Dataset malayalam_news downloaded and prepared to /root/.cache/huggingface/datasets/rajeshradhakrishnan___malayalam_news/malayalam_news/1.0.0/7d6fb88a050b410d95b4f750165a6f797581be76cc65793eb4c6b71ab9ca9b09. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
raw_datasets['train'].features["label"].names

['business', 'entertainment', 'sports', 'technology']

In [ ]:
def addTargeCols(example):
    return {"business": 1 if example["label"] == 0 else 0,
            "entertainment": 1 if example["label"] == 1 else 0,
            "sports": 1 if example["label"] == 2 else 0,
            "technology": 1 if example["label"] == 3 else 0}

In [ ]:
raw_datasets = raw_datasets.map(addTargeCols)

Parameter 'function'=<function addTargeCols at 0x7f60a37995f0> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


  0%|          | 0/4800 [00:00<?, ?ex/s]

  0%|          | 0/600 [00:00<?, ?ex/s]

  0%|          | 0/600 [00:00<?, ?ex/s]

In [ ]:
raw_datasets['train'] = raw_datasets['train'].add_column("is_valid", [False] * len(raw_datasets['train']))
raw_datasets['test'] = raw_datasets['test'].add_column("is_valid", [True] * len(raw_datasets['test']))

In [ ]:
raw_datasets['train'][:2]

{'business': [0, 0],
 'entertainment': [0, 0],
 'is_valid': [False, False],
 'label': [2, 2],
 'sports': [1, 1],
 'technology': [0, 0],
 'text': ['മത്സര പ്രതിഫലമായി സ്വന്തമാക്കിയത് പതിനേഴ് ദശലക്ഷം ഡോളർ . ബാക്കി തുകയത്രയും പരസ്യ വരുമാനത്തിലൂടെ സ്വന്തമാക്കിയതാണ് . ബോൾട്ടിനോളം വിപണിമൂല്യമുള്ള മറ്റൊരു അത്ലറ്റില്ല . ഇതുകൊണ്ടു തന്നെയാണ് ബോൾട്ട് ലോകത്തെ ഏറ്റവും വരുമാനമുള്ള 100 കായിക താരങ്ങളുടെ പട്ടികയിലെ ഏക അത്ലറ്റായത് . പട്ടികയിൽ ഇരുപത്തിമൂന്നാം സ്ഥാനത്താണ് ജമൈക്കൻ ഇതിഹാസം . വിരമിച്ചതിന് ശേഷം ഫുട്ബോളിൽ സജീവമാകാൻ ഒരുങ്ങുന്ന വേഗരാജാവ് ഉസൈൻ ബോൾട്ട് ഫൌണ്ടേഷനും തുടക്കമിട്ടുകഴിഞ്ഞു . പാവപ്പെട്ട കുട്ടികളുടെ വിദ്യാഭ്യാസത്തിനാണ് ഫൌണ്ടേഷൻ പ്രധാനമായും സഹായിക്കുക . എട്ട് ഒളിംപിക് സ്വർണത്തിന് ഉടമയായ ബോൾട്ടിന്റെ പേരിലാണ് 100,200 മീറ്ററുകളിലെ ലോകറെക്കോർഡ് .',
  'ഇന്ത്യൻ പ്രീമിയർ ലീഗിൽ കൊൽക്കത്ത നൈറ്റ് റൈഡേഴ്സിനെതിരേ സൺറൈസേഴ്സിന് 140 റൺസ് വിജയലക്ഷ്യം . ടോസ് നഷ്ടപ്പെട്ട് ആദ്യ ബാറ്റിങ് ആരംഭിച്ച കൊൽത്തക്കയ്ക്ക് എട്ട് വിക്കറ്റ് നഷ്ടത്തിൽ 139 റൺസ് മാത്രമാണ് എടുക്കാൻ സാധിച്ചത് . 49 റൺസെടുത്ത ക്രിസ് ലിന്നാണ് അവരുട

In [ ]:
df = raw_datasets.filter(lambda x: x["entertainment"] == 0)
print(df.num_rows)

  0%|          | 0/5 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

{'train': 3578, 'validation': 455, 'test': 467}


In [ ]:
final_ds = datasets.concatenate_datasets([raw_datasets['train'].shuffle().select(range(1000))
                                        , raw_datasets['test'].shuffle().select(range(200))])
mlnews_df = pd.DataFrame(final_ds)
mlnews_df.head()

,text,label,business,entertainment,sports,technology,is_valid
0,കാഴ്ചക്കാരുടെ തള്ളികയറ്റം ഉണ്ടായതോടെ ലോകത്ത് തന്നെ ആദ്യമായി പ്രദർശനത്തിനു വച്ച സെക്സ് റോബോർട്ടിനു കേടുപാടുകൾ സംഭവിച്ചു . ഓസ്ട്രിയയിൽ നടന്ന ആർട്ട്സ് ഇലക്ട്രോണിക് ഫെസ്റ്റിവെല്ലിലാണ് സമാന്ത എന്നു പേരായ സെക്സ് റോബോർട്ടിനെ പ്രദർശനത്തിന് വച്ചത് . പ്രദർശനത്തിൻറെ പ്രധാന ആകർഷണം സാമന്തയായിരുന്നു . എന്നാൽ കണ്ടും കേട്ടും മനസിലാക്കി പോയ ചിലർ റോബോർട്ടിനെ ഉപയോഗിച്ചു നോക്കുകയായിരുന്നു . ഇതിനെ ഭാഗമായി റോബോർട്ടിൻറെ വിവിധ ഭാഗങ്ങളിൽ ചിലർ അമർത്തിയും വലിച്ചും നോക്കി . ഇതേ തുടർന്നാണു സാമന്തയുടെ ചില ഭാഗങ്ങൾ അടർന്നു പോയത് . കൂടാതെ വിരലും നഷ്ടമായി . 3000 യൂറോ വിലവരുന്ന റോബോർട്ടിനാണു പരിക്കേറ്റത് . കേടുപാടുകൾ പരിഹരി...,3,0,0,0,1,False
1,"ശ്രീലങ്കയ്ക്കെതിരായ കന്നി ടെസ്റ്റിൽ സെഞ്ച്വറി നേടി ഇന്ത്യൻ ക്രിക്കറ്റിലെ പുതിയ താരോദയമായി മാറിയ ഹാർദിക് പാണ്ഡ്യ , അച്ഛൻ ഹിമാൻഷു പാണ്ഡ്യക്ക് നൽകിയത് സർപ്രൈസ് സമ്മാനം . ഹർദിക് പാണ്ഡ്യയും സഹോദരനും മുംബൈ ഇന്ത്യൻസ് താരവുമായ ക്രുനാൽ പാണ്ഡ്യയും ചേർന്ന് ഒരു ചുവന്ന ജീപ്പ് കോംപസ് കാർ ആണ് പിതാവിന് അപ്രതീക്ഷിത സമ്മാനമായി നൽകിയത് . മക്കളുടെ ക്രിക്കറ്റ് ഭാവിക്ക് വേണ്ടി ജീവിതം തന്നെ ഉഴിഞ്ഞവെച്ചയാളാണ് ഹിമാൻഷു പാണ്ഡ്യ . മകൻ നൽകിയ സമ്മാനത്തിന്റെ സന്തോഷത്തിൽ കണ്ണുനിറഞ്ഞു നിൽക്കുന്ന പിതാവിന്റെ ദൃശ്യങ്ങൾ പാണ്ഡ്യ തന്നെയാണ് ട്വിറ്ററിൽ പോസ്റ്റ് ചെയ്തത് . ഒപ്പം ഒരു കുറിപ്പും . ' അച്ഛന്റെ മുഖം ഇങ്ങനെ തെളിഞ്ഞു കാണുന...",2,0,0,1,0,False
2,"ഡബ്ല്യൂസിസി അംഗങ്ങളുടെ ആവശ്യപ്പെട്ടതനുസരിച്ച് അങ്ങോട്ട് ആവശ്യപ്പെട്ടതോടെ അമ്മ സംഘടനയിൽ നിന്ന് നടൻ ദിലീപ് രാജിവെച്ചെന്ന് പ്രസിഡൻറ് മോഹൻലാൽ . താൻ ഈ സ്ഥാനത്ത് എത്തുമ്പോൾ ഏറ്റവും വലിയ വിഷയമായിരുന്നു ദിലീപിൻറെ കാര്യം . ഡബ്ല്യൂസിസി ആവശ്യപ്പെട്ട കാര്യങ്ങളിൽ നിയമസഹായം തേടിയപ്പോൾ ഒരു ജനറൽ ബോഡി കൂടി മാത്രമേ തീരുമാനം എടുക്കാൻ സാധിക്കൂ എന്നാണ് മനസിലായത് . പിന്നീട് അത് വലിയ പ്രശ്നമായി മാറി . ഡബ്ല്യൂസിസി ഇക്കാര്യത്തിൽ വ്യക്തമായ ധാരണ വേണമെന്ന് പറഞ്ഞു . അപ്പോൾ അൽപം സാവകാശം വേണമെന്ന് അവരോട് പറഞ്ഞു . ജനറൽ ബോഡി കൂടി തീരുമാനിക്കാൻ സാധിക്കുകയുള്ളുവെന്നും അറിയിച്ചു . പിന്നീട് , കേരളത്തിൽ പ്രളയം പോലുള്ള പ്രശ്നങ്...",1,0,1,0,0,False
3,സംസ്ഥാനത്തെ എല്ലാ താലൂക്കാശുപത്രികളിലും ഹൃദ്രോഗ ചികിത്സാ വിഭാഗം സ്ഥാപിക്കും . എല്ലാ മെഡിക്കൽ കോളേജുകളിലും ഓങ്കോളജി ഡിപ്പാർട്ട്മെന്റ് സ്ഥാപിക്കും . മലബാർ ക്യാൻസർ സെന്ററിനെ ആർസിസി നിലവാരത്തിലേക്ക് ഉയർത്തും . കൊച്ചിയിൽ ഇതേ സൌകര്യത്തിൽ പുതിയ ആശുപത്രി സ്ഥാപിക്കും . സംസ്ഥാനത്തെ എൺപത് ശതമാനം ക്യാൻസർ രോഗികൾക്കും ചികിത്സ നൽകാനുള്ള ശേഷി സർക്കാർ ആശുപത്രികളിലുണ്ടാവും . 550 ഡോക്ടർമാരേയും 1750 നഴ്സുമാരേയും നിയമിക്കും . വ്യക്തിഗത സൂഷ്മ ആരോഗ്യ പദ്ധതി നടപ്പിലാക്കും . അപകടങ്ങളിൽപ്പെടുന്നവർക്ക് അടിയന്തര ചികിത്സ ഉറപ്പാക്കാൻ പ്രത്യേകസംവിധാനം . ഇവരുടെ ചികിത്സാ ചിലവിനായുള്ള പണം റോഡ് ഫണ്ടിൽ നിന്നും കണ്ടെത്തും . ഊ...,0,1,0,0,0,False
4,"ഇന്ത്യയുടെ ഓൺലൈൻ ടിക്കറ്റ് ബുക്കിംഗ് ഇടമായ ബുക്ക് മൈ ഷോ വാട്ട്സ്ആപ്പുമായി ബന്ധിപ്പിച്ച് ഇടപാട് വിപുലീകരിക്കുന്നു . ടിക്കറ്റിന്റേയും ടിക്കറ്റ് ബുക്കിംഗിന്റേയും വിവരങ്ങൾ ഉപയോക്താക്കൾക്ക് വാട്ട്സ്ആപ്പ് വഴി അയക്കാനാണ് ബുക്ക് മൈ ഷോ പദ്ധതിയിടുന്നത് . ബിസിനസ് സംബന്ധമായി വാട്ട്സ്ആപ്പുമായി ഇടപാട് നടത്തുന്ന ആദ്യത്തെ കമ്പനിയാണ് തങ്ങളെന്ന് ബുക്ക് മൈ ഷോ അവകാശപ്പെടുന്നു . ”രാജ്യത്ത് ലക്ഷക്കണക്കിന് പേർ ആശയവിനിമയത്തിനായി മുൻഗണന നൽകന്നത് വാട്ട്സ്ആപ്പ് ആയി മാറിയിട്ടുണ്ട് . അത് കൊണ്ട് തന്നെയാണ് ടിക്കറ്റ് സംബന്ധമായ വിവരങ്ങൾ കൈമാറാൻ വാട്ട്സ്ആപ്പിനെ തെരഞ്ഞെടുക്കുന്നതിനെ കുറിച്ച് ഞങ്ങൾ ആലോചിച്ചത്” , ബുക്ക് മൈ ഷോ...",3,0,0,0,1,False


# 3. Using Blurr High Level API

In [ ]:
lbl_cols = raw_datasets['train'].features["label"].names
lbl_cols

['business', 'entertainment', 'sports', 'technology']

In [ ]:
from transformers import AutoConfig, AutoModelForSequenceClassification
model_cls = AutoModelForSequenceClassification

pretrained_model_name = "xlm-roberta-base"
config = AutoConfig.from_pretrained(pretrained_model_name)
config.num_labels = len(lbl_cols)

hf_arch, hf_config, hf_tokenizer, hf_model = get_hf_objects(pretrained_model_name, model_cls=model_cls, config=config)
hf_model.config.problem_type = "multi_label_classification"

print(hf_arch)
print(type(hf_config))
print(type(hf_tokenizer))
print(type(hf_model))

Downloading:   0%|          | 0.00/615 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.83M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/8.68M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.04G [00:00<?, ?B/s]

xlm_roberta
<class 'transformers.models.xlm_roberta.configuration_xlm_roberta.XLMRobertaConfig'>
<class 'transformers.models.xlm_roberta.tokenization_xlm_roberta_fast.XLMRobertaTokenizerFast'>
<class 'transformers.models.xlm_roberta.modeling_xlm_roberta.XLMRobertaForSequenceClassification'>


In [ ]:
learn = BlearnerForSequenceClassification.from_data(
    mlnews_df, pretrained_model_name, text_attr="text", label_attr=lbl_cols, dl_kwargs={"bs": 4}
)

learn.loss_func.thresh = 0.02

In [ ]:
learn.fit_one_cycle(1, lr_max=1e-2)

epoch,train_loss,valid_loss,f1_score,accuracy_multi,time
0,0.603728,0.572628,0.000000,0.750000,00:47


In [ ]:
learn.loss_func.thresh = 0.01

comment = """
2032 ഒളിമ്പിക്‌സിന് ബ്രിസ്‌ബെയ്ന്‍ വേദിയാകും; ഗെയിംസിന് വേദിയാകുന്ന മൂന്നാമത്തെ ഓസ്‌ട്രേലിയന്‍ നഗരം
"""
learn.predict(comment)

[{'class_indices': [1, 1, 1, 1],
  'class_labels': ['business', 'entertainment', 'sports', 'technology'],
  'labels': ['business', 'entertainment', 'sports', 'technology'],
  'probs': [0.2227945327758789,
   0.36078402400016785,
   0.2089862823486328,
   0.23871146142482758],
  'scores': [0.2227945327758789,
   0.36078402400016785,
   0.2089862823486328,
   0.23871146142482758]}]

# 5. Share in Huggingface

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

Login successful
Your token has been saved to /root/.huggingface/token
Authenticated through git-credential store but this isn't the helper defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub. Run the following command in your terminal in case you want to set this credential helper as the default

git config --global credential.helper store


In [ ]:
from huggingface_hub import push_to_hub_fastai

push_to_hub_fastai(learner=learn, repo_id=repo_id)

Cloning https://huggingface.co/rajeshradhakrishnan/ml-news-classify-fastai into local empty directory.
Adding files tracked by Git LFS: ['model.pkl']. This may take a bit of time if the files are large.


Upload file model.pkl:   0%|          | 3.33k/1.04G [00:00<?, ?B/s]

To https://huggingface.co/rajeshradhakrishnan/ml-news-classify-fastai
   bfea4f0..bbe4199  main -> main



'https://huggingface.co/rajeshradhakrishnan/ml-news-classify-fastai/commit/bbe41991bc2c4e045ed50e0e7321ad5b4fd5530f'

# 6. Loading a Learner from the Hugging Face Hub

In [ ]:
from huggingface_hub import from_pretrained_fastai
# ഞാൻ ചിന്തിക്കുന്ന കുറച്ചു ആശയങ്ങൾ പ്രാവർത്തികം ആക്കുക എന്നുള്ളതാണ്
# ഇതുവരെ ഞാൻ കണ്ടുമുട്ടിയതും പരിചയവുമുള്ള സോഫ്റ്റ്‌വെയർ സേവനങ്ങൾ 
# എങ്ങനെ പ്രയോജനപ്പെടുത്താം എന്നുള്ള വിവരങ്ങൾ ഇവിടെ പരിചയ പെടുത്തുന്നതുമാണ്. 
learner = from_pretrained_fastai(repo_id)

Downloading:   0%|          | 0.00/1.22k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.78k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/160 [00:00<?, ?B/s]

In [ ]:
sentences = ["ഓഹരി വിപണി തകരുമ്പോള്‍ നിക്ഷേപം എങ്ങനെ സുരക്ഷിതമാക്കാം",
             "വാര്‍ണറുടെ ഒറ്റക്കയ്യന്‍ ക്യാച്ചില്‍ അമ്പരന്ന് ക്രിക്കറ്റ് ലോകം"]

probs = learner.predict(sentences)
# 'business', 'entertainment', 'sports', 'technology'
for idx in range(len(sentences)):
  print(f"Probability that sentence '{sentences[idx]}' is business is: {100*probs[idx]['probs'][0]:.2f}%")
  print(f"Probability that sentence '{sentences[idx]}' is entertainment is: {100*probs[idx]['probs'][1]:.2f}%")
  print(f"Probability that sentence '{sentences[idx]}' is sports is: {100*probs[idx]['probs'][2]:.2f}%")
  print(f"Probability that sentence '{sentences[idx]}' is technology is: {100*probs[idx]['probs'][3]:.2f}%")

Probability that sentence 'ഓഹരി വിപണി തകരുമ്പോള്‍ നിക്ഷേപം എങ്ങനെ സുരക്ഷിതമാക്കാം' is business is: 22.39%
Probability that sentence 'ഓഹരി വിപണി തകരുമ്പോള്‍ നിക്ഷേപം എങ്ങനെ സുരക്ഷിതമാക്കാം' is entertainment is: 24.75%
Probability that sentence 'ഓഹരി വിപണി തകരുമ്പോള്‍ നിക്ഷേപം എങ്ങനെ സുരക്ഷിതമാക്കാം' is sports is: 29.82%
Probability that sentence 'ഓഹരി വിപണി തകരുമ്പോള്‍ നിക്ഷേപം എങ്ങനെ സുരക്ഷിതമാക്കാം' is technology is: 25.26%
Probability that sentence 'വാര്‍ണറുടെ ഒറ്റക്കയ്യന്‍ ക്യാച്ചില്‍ അമ്പരന്ന് ക്രിക്കറ്റ് ലോകം' is business is: 22.39%
Probability that sentence 'വാര്‍ണറുടെ ഒറ്റക്കയ്യന്‍ ക്യാച്ചില്‍ അമ്പരന്ന് ക്രിക്കറ്റ് ലോകം' is entertainment is: 24.75%
Probability that sentence 'വാര്‍ണറുടെ ഒറ്റക്കയ്യന്‍ ക്യാച്ചില്‍ അമ്പരന്ന് ക്രിക്കറ്റ് ലോകം' is sports is: 29.82%
Probability that sentence 'വാര്‍ണറുടെ ഒറ്റക്കയ്യന്‍ ക്യാച്ചില്‍ അമ്പരന്ന് ക്രിക്കറ്റ് ലോകം' is technology is: 25.26%


# 7. Create a Gradio demo to showcase your model

In [1]:
!pip install -Uqq gradio

     |████████████████████████████████| 5.1 MB 5.2 MB/s 
     |████████████████████████████████| 256 kB 73.8 MB/s 
     |████████████████████████████████| 84 kB 3.9 MB/s 
     |████████████████████████████████| 54 kB 3.7 MB/s 
     |████████████████████████████████| 140 kB 76.2 MB/s 
     |████████████████████████████████| 212 kB 66.6 MB/s 
     |████████████████████████████████| 53 kB 3.1 MB/s 
     |████████████████████████████████| 2.0 MB 59.7 MB/s 
     |████████████████████████████████| 1.1 MB 52.7 MB/s 
     |████████████████████████████████| 94 kB 4.9 MB/s 
     |████████████████████████████████| 271 kB 73.6 MB/s 
     |████████████████████████████████| 144 kB 74.8 MB/s 
     |████████████████████████████████| 11.1 MB 54.2 MB/s 
     |████████████████████████████████| 63 kB 2.6 MB/s 
     |████████████████████████████████| 80 kB 12.0 MB/s 
     |████████████████████████████████| 43 kB 2.9 MB/s 
     |████████████████████████████████| 4.0 MB 52.1 MB/s 
     |█████████████████████

In [7]:
from pathlib import Path
import gradio as gr
from huggingface_hub import from_pretrained_fastai


LABELS = Path('class_names.txt').read_text().splitlines()



def predict(news_headline):
    learner = from_pretrained_fastai("rajeshradhakrishnan/ml-news-classify-fastai")

    probabilities = learner.predict(news_headline)

    return {LABELS[i]: probabilities[0]['probs'][i] for i in range(len(LABELS))}

interface = gr.Interface(
    predict, 
    inputs="textbox", 
    outputs='label', 
    theme="huggingface", 
    title="Malayalam News Classifier", 
    description="Try to classify news in മലയാളം? Input a few malayalam news headlines and verify whether the model categorized it appropriately!", 
    article = "<p style='text-align: center'>Malayalam News Classifier | Demo Model</p>",
    examples=[["ഓഹരി വിപണി തകരുമ്പോള്‍ നിക്ഷേപം എങ്ങനെ സുരക്ഷിതമാക്കാം"], ["വാര്‍ണറുടെ ഒറ്റക്കയ്യന്‍ ക്യാച്ചില്‍ അമ്പരന്ന് ക്രിക്കറ്റ് ലോകം"]],
    # live=True,
    share=True)
interface.launch(debug=True)


/usr/local/lib/python3.7/dist-packages/gradio/deprecation.py:44: UserWarning: You have unused kwarg parameters in Interface, please remove them: {'share': True}
  f"You have unused kwarg parameters in {cls}, please remove them: {kwargs}"
/usr/local/lib/python3.7/dist-packages/gradio/interface.py:286: UserWarning: Currently, only the 'default' theme is supported.
  warnings.warn("Currently, only the 'default' theme is supported.")


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://34892.gradio.app

This share link expires in 72 hours. For free permanent hosting, check out Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.


(<gradio.routes.App at 0x7f42d9a1ba10>,
 'http://127.0.0.1:7860/',
 'https://34892.gradio.app')

In [8]:
gr.Interface.load("spaces/rajeshradhakrishnan/malayalam-news-classify").launch()

Fetching interface from: https://huggingface.co/spaces/rajeshradhakrishnan/malayalam-news-classify


TypeError: ignored

---
tags:
- fastai
---

# Amazing!

🥳 Congratulations on hosting your fastai model on the Hugging Face Hub!

# Some next steps
1. Fill out this model card with more information (see the template below and the [documentation here](https://huggingface.co/docs/hub/model-repos))!

2. Create a demo in Gradio or Streamlit using 🤗 Spaces ([documentation here](https://huggingface.co/docs/hub/spaces)).

3. Join the fastai community on the [Fastai Discord](https://discord.com/invite/YKrxeNn)!

Greetings fellow fastlearner 🤝! Don't forget to delete this content from your model card.


---


# Model card

## Model description
More information needed

## Intended uses & limitations
More information needed

## Training and evaluation data
More information needed
